In [1]:
%%writefile 'config.py'


token = 'сюда свой токет для бота'

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = '''
Ты — Сайга, русскоязычный автоматический ассистент. 
Ты разговариваешь с людьми и помогаешь им.'''


# Праметры подключения к базе PostgreSQL
dbname = 'db_bot'
user = 'db_user'
password = 'password'
host = 'localhost'
host = '127.0.0.1'

pdf_path = '../pdf/'
txt_patx = '../docs/'
num_q = 5

Overwriting config.py


In [2]:
from model_gpu import *

model_qa

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [3]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from config import *
import joblib
from sentence_transformers import SentenceTransformer
import torch
import json
from psycopg2.errorcodes import UNIQUE_VIOLATION
from psycopg2 import errors

connection = psycopg2.connect(
    dbname = dbname,
    user = user, 
    password = password, 
    host = host
)
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = connection.cursor()

sql = 'SELECT version();'

cursor.execute(sql)
records = cursor.fetchall()
print(records)

[('PostgreSQL 16.1 (Ubuntu 16.1-1.pgdg22.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)]


In [4]:
sql = 'GRANT USAGE ON SCHEMA public TO db_user;'
cursor.execute(sql)
sql = 'DROP TABLE IF EXISTS embedding;'
cursor.execute(sql)
sql = 'DROP TABLE IF EXISTS documents;'
cursor.execute(sql)

In [5]:
sql = '''
CREATE TABLE IF NOT EXISTS embedding 
(id SERIAL PRIMARY KEY, emb vector(1024), question TEXT UNIQUE, answer TEXT, theme TEXT);
'''
      
cursor.execute(sql)

In [6]:
sql = '''
CREATE TABLE IF NOT EXISTS documents 
(id SERIAL PRIMARY KEY, emb_theme vector(1024), text_doc TEXT UNIQUE, theme_doc TEXT);
'''
cursor.execute(sql)

In [7]:
from sentence_transformers import SentenceTransformer
import torch
import json
import random
import numpy as np

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

embeddings_model = SentenceTransformer(
    #'sentence-transformers/stsb-roberta-large',
    'intfloat/multilingual-e5-large',
    device = 'cpu'
)

def get_embedding(sentence):    
    embeddings = embeddings_model.encode(sentence)
    return embeddings

In [8]:
text_docs = 'Добрый день. Здравствуйте. Как дела?'
for text_doc in text_docs.split('. '):
    emb = get_embedding(text_doc)
    emb_theme = json.dumps(emb.tolist())
    answer = 'Добрый день.'
    cursor.execute(
            "INSERT INTO embedding (emb, question, answer, theme) VALUES (%s, %s, %s, %s)", 
            (emb_theme, text_doc, answer, text_doc)
        )

In [9]:
text_docs = 'До свидания. Пока. Гуд бай'
for text_doc in text_docs.split('. '):
    emb = get_embedding(text_doc)
    emb_theme = json.dumps(emb.tolist())
    answer = 'До свидания.'    
    try:
        cursor.execute(
            "INSERT INTO embedding (emb, question, answer, theme) VALUES (%s, %s, %s, %s)", 
            (emb_theme, text_doc, answer, text_doc)
            )
    except errors.lookup(UNIQUE_VIOLATION) as e:
        print(text_doc, e)

In [10]:
import os
from config import *
from pyaspeller import YandexSpeller
import re
speller = YandexSpeller()

files_txt = sorted(os.listdir(txt_patx))
files_txt

['01.txt', '02.txt', '03.txt', '04.txt', '05.txt', '06.txt', '07.txt']

In [11]:
for file in files_txt:
    
    # чтение файла
    with open(txt_patx + file) as file:
        lines = [line.rstrip() for line in file]
    # чтение файла
    
    # строки в текст и исправление слов с ошибками
    text_doc = ' '.join(lines)
    text_doc = speller.spelled(text_doc)    
    text_doc = text_doc.replace('\xa0', '')
    text_doc = text_doc.replace('\n', ' ')
    text_doc = text_doc.replace('.', '. ')
    text_doc = text_doc.replace('...', '... ')
    text_doc = text_doc.replace(',', ', ')
    text_doc = text_doc.replace('!', '! ')
    text_doc = text_doc.replace('  ', ' ')
    text_doc = text_doc.replace('   ', ' ')
    text_doc = text_doc.replace('    ', ' ')
    #text_docs = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text_doc)
    
    promnt = '<s>system\nСократи текст до одного предложения.</s>'
    theme_doc = model_qa.predict_answer(promnt + '###\n' + text_doc + '\n###')
        
    emb = get_embedding(theme_doc)
    emb_theme = json.dumps(emb.tolist())
    
    try:
        cursor.execute(
            "INSERT INTO documents (emb_theme, text_doc, theme_doc) VALUES (%s, %s, %s)", 
            (emb_theme, text_doc, theme_doc)
        )
    except errors.lookup(UNIQUE_VIOLATION) as e:
        print(text_doc, e)
        
    promnt = f'<s>system\nПридумай {num_q} вопросов к тексту.</s>'    
    temp = model_qa.predict_answer(promnt + '###\n' + text_doc + '\n###')
    temp = temp.split('\n')
    questions = []
    for q in temp:
        q = speller.spelled(q)
        questions.append(q)        
        
    promnt = '<s>system\nОтветь кратко на вопрос. Не более 100 слов.</s>'
    for i, question in enumerate(questions):
        
        #for z in range(num_q + 1):        
        #    question = question.replace(f'{z}. ', '')
            
        question = question.replace(f'{i + 1}. ', '')

        print('Вопрос:', question)
        answer = model_qa.predict_answer(promnt + text_doc + question)
        answer = speller.spelled(answer)
        print('Ответ:', answer)
        emb = get_embedding(question)
        emb = json.dumps(emb.tolist())
        
        try:
            cursor.execute(
                "INSERT INTO embedding (emb, question, answer, theme) VALUES (%s, %s, %s, %s)", 
                (emb, question, answer, theme_doc)
            )
        except errors.lookup(UNIQUE_VIOLATION) as e:
            print(text_doc, e)
        
        print('\n')
    
    #####

/home/vlad/anaconda3/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
2023-12-15 21:03:51.022569: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 21:03:51.685339: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 21:03:51.685385: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft

Вопрос: Какое количество этажей может быть построено на гаражах в городе Мирном?
Ответ: Максимальное количество этажей, которое может быть построено на гаражах в городе Мирном, согласно Правилам землепользования и землеустройства, составляет 3 этажа.


Вопрос: Какие действия должны быть предприняты для сноса старых гаражей?
Ответ: Для сноса старых гаражей, которые расположены на территории города Мирного, следует выполнить несколько шагов:

1. Получить разрешение на снос здания от органов местной власти. Для этого необходимо подать заявление с просьбой о разрешении на снос здания.
2. Определить стоимость работ по сносу здания и найти компанию или специалистов, которые могут выполнить эти работы.
3. Нанять специалистов для проведения геотехнических исследований на месте будущих работ, чтобы определить возможные проблемы и риски при сносе здания.
4. Убедиться в том, что все работы по сносу здания будут проводиться в соответствии с требованиями местных норм и правил безопасности.
5. После

Ответ: Уборкой снега на придомовых территориях многокомнатных домов, перед подъездами и на троttoарах по улице Энтузиастов занимается управляющая организация МУП "АПЖХ".


Вопрос: В какие месяцы года уборка снега происходит?
Ответ: Уборка снега на придомовых территориях многоквартирных домов, перед подъездами и на тротуарах по ул. Энтузиастов осуществляется в марте и декабре согласно утвержденному тарифу. Однако, в связи с обильными объемами снегопада, УК неоднократно производила очистку в апреле и мае 2023 года.


Вопрос: Что делает управляющая компания МУП "ОПЖКХ"?
Ответ: МУП "ОП ЖКХ" занимается уборкой снега над придомовых территориях многоэтажек, перед подъездами и над тротуаром по улице Энтузиастов. В соответствии с утвержденным тарифом, очистка от снега проводится два раза в год - в марте и декабре. Однадко, из-за большого количества выпадающего снега, УК проводила очистку также в апреле и мае 2023 года. Кроме того, планируется вывоз брошенных транспортных средств, включая бесхоз

Ответ: Чтобы расторгнуть договор, вам необходимо написать заявление о расторжении договора. Заявление должно быть подписано и датировано, а также содержать информацию об условиях, которые вы хотите изменить или прекратить. Заявление можно отправить администрации спорткомплекса.


Вопрос: Как рассчитывается сумма возврата при расторжении договора?
Ответ: Сумма возврата при расторжении договора рассчитывается согласно Правил оказания физкультурных услуг и посещения спортивных объекторов Культурно-Спортивного Комплекса АК "АЛРОСА". Сумма возврата составляет разницу между стоимостью услуги и суммой, которую клиент оплатил за услугу. Если клиент оплатил услугу полностью, то возврат составит нулевую сумму. Если клиент оплатил только часть стоимости услуги, то возврат составит разницу между оплаченной и неоплаченной суммой. Возврат осуществляется в течение 10 рабочих дней после получения заявления о расторжении договора.


